In [1]:
import os
import pandas as pd
import geopandas as gpd
import xarray as xr
import numpy as np
import itertools
import datetime
from shapely.geometry import GeometryCollection, Point, LineString
from lotek.conversion import shp2mask
from lotek.gps import make_steps, graze_intensity
from rasterio.enums import MergeAlg
from tqdm.notebook import tqdm
import pyproj
import math

In [2]:
"""
Setup a cluster for parallel processing
"""
from distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=5, threads_per_worker=2)
client = Client(cluster)

In [3]:
"""
Setup for calculating bearing of steps
"""
geodesic = pyproj.Geod(ellps='WGS84')
utm_13N = pyproj.Proj(32613)

In [4]:
"""
Define I/O
"""
# set the year of analysis
year = 2016

# Inputs
# path to main directory
inDIR = 'C:/SPK_local/for_others/Augustine_lotek'
# file name of cleaned gps data (from gps_cleaning.ipynb)
gps_f = 'TRMappended_GoodDays_' + str(year) + 'TrackQ3_cleaned.csv'
# path to 1 m vegetation raster as grid template
veg_f = 'G:/neon_v18/neon_class_2017_v18.tif'
# path to 30 m raster as grid template
tif_30m_f = 'C:/SPK_local/data/rasters/Masks/CPER_dist_to_fence_2017.tif'
# path to shapefile with pasture boundaries
cper_f = "C:/SPK_local/data/vectors/Pasture_Boundaries/Shapefiles/cper_pastures_2017_clip.shp"

# Outputs
# path to output directory
outDIR = 'C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/data'
# file name of gridded output
out_gridded_f = str(year) + '_grazing_time_gridded_all.csv'

In [5]:
"""
Read in data and pre-format
"""
# read in GPS data
df_gps = pd.read_csv(os.path.join(inDIR, gps_f), engine='python', parse_dates=['Fix_Date', 'Fix_DateTime'])

# read in CPER pasture shapefile
df_cper = gpd.read_file(cper_f)

# dissolve by pasture to make sure each pasture is a single polygon
df_cper = df_cper.dissolve(by='Past_Name_').reset_index().rename(columns={'Past_Name_': 'Pasture'})

# convert pasture polygons to an xarray (1 m gridded raster) object with pasture name as value
cper_info = df_cper[['Pasture', 'geometry']].reset_index(drop=True).reset_index().rename(columns={'index': 'id'})
past_dict = {row.id+1: row.Pasture for _, row in cper_info.iterrows()}
past_dict[0] = 'UNK'
past_mask_shp = [(row.geometry, row.id+1) for _, row in cper_info.iterrows()]

In [6]:
"""
Create daily and weekly groupings of GPS data
"""
# create unique ID by Pasture-Steer combo
df_gps['Pasture_Steer_ID'] = df_gps['Pasture'] + '_' + df_gps['Steer_ID']

# create daily grouping and count number of unique Pastures, Steers and Pasture-Steer combos on each day
df_gps_daily = df_gps.groupby(
    'Fix_Date').agg({'Pasture': lambda x: x.unique(),
                     'Steer_ID': lambda x: x.unique(),
                     'Pasture_Steer_ID': lambda x: x.unique()}).reset_index()

# create weekly grouping listing all pastures, Steers and Pasture-Steer combos present all days in the week
df_gps_wkly = df_gps_daily.groupby(
    df_gps_daily['Fix_Date'].dt.isocalendar().week).agg(
    {'Pasture': lambda x: [i for i in list(df_gps['Pasture'].unique()) if all(i in y for y in x)],
     'Steer_ID': lambda x: [i for i in list(df_gps['Steer_ID'].unique()) if all(i in y for y in x)],
     'Pasture_Steer_ID': lambda x: [i for i in list(df_gps['Pasture_Steer_ID'].unique()) if all(i in y for y in x)]}).reset_index()

In [7]:
type(df_gps_daily['Pasture'][0])

str

In [8]:
"""
Print out all combos of Pastures available for each week across a set range of weeks
"""
reqd_pastures = []
min_n_pastures = 5
for r in range(min_n_pastures, 10):
    past_combos = list(itertools.combinations(df_gps['Pasture'].unique(), r))
    past_combos_sum = np.array(
        [sum(all(elem in test2 for elem in test1) for test2 in df_gps_wkly['Pasture']) for test1 in past_combos])
    top_combos = np.array(past_combos)[(past_combos_sum == np.max(past_combos_sum)) | (past_combos_sum > 5)]
    if len(reqd_pastures) > 0:
        reqd_mask = [all(p in x for p in reqd_pastures) for x in top_combos]
        top_combos = top_combos[reqd_mask]
    top_combos = [list(x) for x in top_combos]
    top_dates = []
    for combo in top_combos:
        mask = [all(j in i for j in combo) for i in df_gps_wkly['Pasture']]
        top_dates.append(
            [datetime.date.fromisocalendar(year, x, 1).strftime('%b %d') for x in df_gps_wkly['week'][mask].values])
    print('\n\n' + '-------------------------------------' + '\n' + 
          'No. Pastures: ' + str(r) + '\n'
          'Max no. weeks: ' + str(np.max(past_combos_sum)) + '\n' +
          '-------------------------------------')
    for dates, combo in [(y, x) for y, x in sorted(zip(top_dates, top_combos), key=lambda pair: len(pair[0]), reverse=True)]:
        print('Pastures=' + str(len(combo)) + ': ' + str(combo))
        past_mask = [all(j in i for j in combo) for i in df_gps_wkly['Pasture']]
        #steer_mask = [all([j in y] for y in i for j in combo) for i in df_gps_wkly['Pasture_Steer_ID']]
        steer_ct = [sum([sum([j in y for j in combo]) for y in x]) for x in df_gps_wkly['Pasture_Steer_ID'][past_mask]]
        #steers = [len(x) for x in df_gps_wkly['Pasture_Steer_ID'][steer_mask]]
        #dates = [datetime.date.fromisocalendar(2017, x, 1).strftime('%B %d') for x in df_gps_wkly['week'][mask].values]
        print('Weeks=' + str(len(dates)) + ' (start dates): ' + str(dates))
        print('Steers: ' + str(steer_ct) + '\n')




-------------------------------------
No. Pastures: 5
Max no. weeks: 18
-------------------------------------
Pastures=5: ['5E', '7NW', '17N', '19N', '20SE']
Weeks=18 (start dates): ['May 16', 'May 23', 'May 30', 'Jun 06', 'Jun 13', 'Jun 20', 'Jul 04', 'Jul 11', 'Jul 18', 'Jul 25', 'Aug 01', 'Aug 08', 'Aug 15', 'Aug 22', 'Sep 05', 'Sep 12', 'Sep 19', 'Sep 26']
Steers: [9, 11, 11, 11, 10, 11, 10, 11, 11, 10, 10, 11, 11, 11, 9, 10, 11, 11]

Pastures=5: ['5E', '7NW', '17N', '19N', '31E']
Weeks=18 (start dates): ['May 16', 'May 23', 'May 30', 'Jun 06', 'Jun 13', 'Jun 20', 'Jul 04', 'Jul 11', 'Jul 18', 'Jul 25', 'Aug 01', 'Aug 08', 'Aug 15', 'Aug 22', 'Sep 05', 'Sep 12', 'Sep 19', 'Sep 26']
Steers: [10, 11, 11, 11, 10, 11, 9, 11, 11, 10, 10, 11, 11, 11, 8, 9, 10, 10]

Pastures=5: ['5E', '7NW', '17N', '20SE', '31E']
Weeks=18 (start dates): ['May 16', 'May 23', 'May 30', 'Jun 06', 'Jun 13', 'Jun 20', 'Jul 04', 'Jul 11', 'Jul 18', 'Jul 25', 'Aug 01', 'Aug 08', 'Aug 15', 'Aug 22', 'Sep 05', '

In [9]:
"""
Set the final combo of training pastures for modelling and create mask for extracting weeks when all are available
"""
# specific final pastures
fnl_pastures = ['7NW', '17N', '20SE', '15E', '26E', '31E']
#fnl_pastures = ['7NW', '17N', '24W', '26E', '25SE', '31E']

# create mask of final weeks based on pastures
wk_mask = [all(j in i for j in fnl_pastures) for i in df_gps_wkly['Pasture']]

# check which weeks are in final dataset
df_gps_wkly['week'][wk_mask].values

array([21, 22, 23, 25, 27, 29, 30, 31, 32, 33, 34, 37, 38, 39],
      dtype=int64)

In [10]:
# read in CPER pasture shapefile
df_cper = gpd.read_file(cper_f)

# dissolve by pasture to make sure each pasture is a single polygon
df_cper = df_cper.dissolve(by='Past_Name_').reset_index().rename(columns={'Past_Name_': 'Pasture'})

# convert pasture polygons to an xarray (1 m gridded raster) object with pasture name as value
cper_info = df_cper[['Pasture', 'geometry']].reset_index(drop=True).reset_index().rename(columns={'index': 'id'})
past_dict = {row.id+1: row.Pasture for _, row in cper_info.iterrows()}
past_dict[0] = 'UNK'
past_mask_shp = [(row.geometry, row.id+1) for _, row in cper_info.iterrows()]

In [11]:
"""
Create 1 m and 30 m raster template for subsequent analysis
"""
# read in 1 m vegetation community raster as template and create pasture mask at 1 m
xr_veg = xr.open_rasterio(veg_f).squeeze('band')
past_mask_1m = shp2mask(shp=past_mask_shp, xr_object=xr_veg)
past_mask_1m.values = np.array([past_dict[i] for i in past_mask_1m.values.flatten()]).reshape(past_mask_1m.shape)

# read in 30 m template and create pasture mask at 30 m
xr_30m = xr.open_rasterio(tif_30m_f).squeeze().reset_coords(drop=True)
past_mask_30m = shp2mask(shp=past_mask_shp, xr_object=xr_30m)
past_mask_30m.values = np.array([past_dict[i] for i in past_mask_30m.values.flatten()]).reshape(past_mask_30m.shape)

In [12]:
"""
Loop through final pastures and weeks to extract final dataset 
of grazing time per cell for a given pasture, steer and time period
"""

# create new column in gps dataset for week-of-year
df_gps['week'] = df_gps['Fix_Date'].dt.isocalendar().week

# create DataFrame template for saving data
df_wkly_grid = pd.DataFrame(columns=['mod_data', 'week', 'Pasture', 'Steer_ID', 'UTM_X', 'UTM_Y', 'grazing_secs'])

# loop through week and pasture and steer to create each dataset
for wk in tqdm(df_gps_wkly['week'].unique()):

#for wk in df_gps_wkly['week'][wk_mask].values:
    print('-------------' +  
          'Week starting: ' + 
          datetime.date.fromisocalendar(2017, wk, 1).strftime('%b %d') + ' (' + str(wk) +
         ') -------------')
    for past in df_gps['Pasture'][df_gps['week'] == wk].unique():
        # check if pasture+week is in training dataset
        if past in fnl_pastures and wk in df_gps_wkly['week'][wk_mask].values:
            mod_data = 'train'
        else:
            mod_data = 'test'
        steer_mask = [past in i for i in df_gps_wkly['Pasture_Steer_ID'][df_gps_wkly['week'] == wk].iloc[0]]
        steer_list = np.array(df_gps_wkly['Steer_ID'][df_gps_wkly['week'] == wk].iloc[0])[steer_mask]
        if len(steer_list) > 0:
            print('Pasture: ' + past + ' (' + mod_data + ')')
        for steer in steer_list:
            print('  ' + steer)
            
            geometry_steps = df_gps[(df_gps.week == wk) &
                                    (df_gps.Pasture == past) &
                                    (df_gps.Steer_ID == steer) &
                                    (df_gps.GrazingAct == 1) &
                                    df_gps.moverate.notnull()].apply(make_steps,
                                                                     start_field_x='UTM_X_fnl_lag1',
                                                                     start_field_y='UTM_Y_fnl_lag1',
                                                                     axis=1)
            if len(geometry_steps) == 0:
                print("    SKIPPING STEER: All points masked.")
                continue            
            else:
                gdf_steps = gpd.GeoDataFrame(df_gps[(df_gps.week == wk) &
                                                (df_gps.Pasture == past) &
                                                (df_gps.Steer_ID == steer) &
                                                (df_gps.GrazingAct == 1) &
                                                df_gps.moverate.notnull()],
                                         geometry=geometry_steps)
                
                steps_coords = gdf_steps.apply(lambda x: utm_13N(x.geometry.coords.xy[0],
                                                 x.geometry.coords.xy[1], 
                                                 inverse=True), axis=1)

                fwd_azimuth_all = steps_coords.apply(lambda x: geodesic.inv(x[0][0],
                                                                            x[1][0],
                                                                            x[0][1],
                                                                            x[1][1])[0])

                gdf_steps['bearing'] = fwd_azimuth_all
                gdf_steps['diff45'] = abs(round(abs(fwd_azimuth_all/90)) - abs(fwd_azimuth_all/90)) * 2.0
                gdf_steps['cell_ct'] = gdf_steps.apply(lambda x: 1 + np.round(
                    x.steplength*math.cos(math.radians(45*x.diff45))), 
                                                       axis=1)

                secs_m = gdf_steps.apply(graze_intensity, axis=1)

                # add movement rate to geodata frame
                gdf_steps['secs_m'] = secs_m.values

                xr_temp_1m = past_mask_1m.where(past_mask_1m == past, drop=True)
                xr_temp_1m = xr_temp_1m.assign_attrs({'transform': (1.0, 0.0, xr_temp_1m.x.min().values - 0.5, 
                                                           0.0, -1.0, xr_temp_1m.y.max().values + 0.5)})
                steps_info = gdf_steps[['secs_m', 'geometry']].reset_index(drop=True).reset_index().rename(
                    columns={'index': 'id'})
                steps_shp = [(row.geometry, row.secs_m) for _, row in steps_info.iterrows()]
                xr_steps_1m = shp2mask(shp=steps_shp, xr_object=xr_temp_1m, dtype='float32',
                                    merge_alg=MergeAlg('ADD'), all_touched=False)#.stack(z=['x', 'y'])
                xr_temp_30m = past_mask_30m.where(past_mask_30m == past, drop=True)
                xr_temp_30m = xr_temp_30m.assign_attrs({'transform': (30.0, 0.0, xr_temp_30m.x.min().values - 15.0, 
                                                           0.0, -30.0, xr_temp_30m.y.max().values + 15.0)})

                xr_interp_temp = xr_temp_30m.to_dataset(name='temp')
                xr_interp_temp = xr_interp_temp.assign(x_30m = xr_interp_temp.x)
                xr_interp_temp = xr_interp_temp.assign(y_30m = xr_interp_temp.y)
                xr_interp_temp = xr_interp_temp.interp_like(xr_steps_1m, method='nearest')
                xr_interp_temp= xr_interp_temp.assign(gi=xr_steps_1m)#.groupby(['x_30m', 'y_30m']).sum()

                xr_steps_30m = xr_interp_temp.set_coords(['x_30m', 'y_30m']).swap_dims(
                    {'x': 'x_30m',
                     'y': 'y_30m'}).stack(
                    points=['x_30m', 'y_30m']).groupby('points').sum().unstack().rename(
                    {'points_level_0': 'x', 'points_level_1': 'y'}).transpose().sortby(
                    'y', ascending=False)['gi'].where(xr_temp_30m == past)
              
                df_tmp = pd.DataFrame({
                    'mod_data': mod_data,
                    'week': wk,
                    'Pasture': past,
                    'Steer_ID': steer,
                    'UTM_X': [i[0] for i in xr_steps_30m.stack(z=['x', 'y'])['z'].values],
                    'UTM_Y': [i[1] for i in xr_steps_30m.stack(z=['x', 'y'])['z'].values],
                    'grazing_secs': xr_steps_30m.stack(z=['x', 'y']).values
                })
                
                df_tmp.loc[df_tmp['grazing_secs'] > 3.40e38, 'grazing_secs'] = np.nan
                df_tmp = df_tmp[df_tmp['grazing_secs'].notnull()]
                print('GPS grazing hrs: ' + str(round(gdf_steps['Fix_Duration'].sum()/60, 2)))
                print('Grid (1 m) grazing hrs: ' + str(round(xr_steps_1m.sum().values/60/60, 2)))
                print('Grid (30 m) grazing hrs: ' + str(round(df_tmp.grazing_secs.sum()/60/60, 2)))

                df_wkly_grid = df_wkly_grid.append(df_tmp)

  0%|          | 0/21 [00:00<?, ?it/s]

-------------Week starting: May 08 (19) -------------
Pasture: 24W (test)
  3770_D1_2016
GPS grazing hrs: 38.42
Grid (1 m) grazing hrs: 38.4
Grid (30 m) grazing hrs: 34.16
  3771_D1_2016
GPS grazing hrs: 42.48
Grid (1 m) grazing hrs: 42.45
Grid (30 m) grazing hrs: 34.59
-------------Week starting: May 15 (20) -------------
Pasture: 5E (test)
  2285_D1_2016
GPS grazing hrs: 76.24
Grid (1 m) grazing hrs: 76.28
Grid (30 m) grazing hrs: 69.28
  2286_D1_2016


distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


GPS grazing hrs: 63.21
Grid (1 m) grazing hrs: 63.31
Grid (30 m) grazing hrs: 52.98
Pasture: 7NW (test)
  2287_D1_2016


distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


...

GPS grazing hrs: 51.36
Grid (1 m) grazing hrs: 51.43
Grid (30 m) grazing hrs: 48.3
Pasture: 19N (test)
  4597_D3_2016


distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

GPS grazing hrs: 46.74
Grid (1 m) grazing hrs: 46.73
Grid (30 m) grazing hrs: 43.4
Pasture: 31E (train)
  4598_D3_2016


distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 31% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 31% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 31% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

GPS grazing hrs: 42.61
Grid (1 m) grazing hrs: 42.66
Grid (30 m) grazing hrs: 40.9


In [14]:
"""
Save dataset to csv
"""
df_wkly_grid.to_csv(os.path.join(outDIR, out_gridded_f), index_label='index_id')